In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
import os

In [2]:
cv=LeaveOneOut()

In [3]:
#X=pd.read_csv("Kmer4.csv",header=None)
y=pd.read_csv("label.csv",header=None)
y=np.array(y)
#print(X.shape)

In [ ]:
def my_classifier1(X,y):
    print(datetime.datetime.now())
    param1=["best","random"]
    param_grid={"splitter":param1}
    grid_search=GridSearchCV(DecisionTreeClassifier(random_state=10),param_grid,scoring="accuracy",cv=cv)
    grid_search.fit(X,y)
    return grid_search.best_params_["splitter"]
def my_classifier2(p1,X,y):
    print(datetime.datetime.now())
    param2=np.array(range(3,10,1))
    param_grid={"max_depth":param2}
    grid_search=GridSearchCV(DecisionTreeClassifier(random_state=10,splitter=p1),param_grid,scoring="accuracy",cv=cv)
    grid_search.fit(X,y)
    return grid_search.best_params_["max_depth"]
def my_classifier(p1,p2,X, y):
    print(datetime.datetime.now())
   # param2=["best","random"]   ##设置splitter范围
   # param3=np.array(range(3,10,1))   ##设置max_depth范围
    i=X.shape[1]
    param3=np.array(range(10,i,100))  ## 设置max_features范围
    param_grid={"max_features":param3}
    grid_search=GridSearchCV(DecisionTreeClassifier(random_state=10,splitter=p1,max_depth=p2),param_grid,scoring="accuracy",cv=cv)
    grid_search.fit(X,y)
#     print(datetime.datetime.now())
#     print("best parameters:{}".format(grid_search.best_params_))                        #输出最优网格化参数
#     print("best cross-validation score:{:.3f}".format(grid_search.best_score_))         #成员提供优化过程中观察到的最好评分
#     print("best estimator:\n{}".format(grid_search.best_estimator_))
    Predict_Y=[]    #存放预测值
    Prob_Y=[]       #存放预测概率
    Really_Y=y  #存放真实标签
    for train_index,test_index in cv.split(X):
        estimator=DecisionTreeClassifier(random_state=10,splitter=p1,max_depth=p2,max_features=grid_search.best_params_["max_features"])
        estimator.fit(X[train_index],y[train_index])
        Predict_Y.append(estimator.predict(X[test_index])[0])
        Prob_Y.append(estimator.predict_proba(X[test_index]))
    Prob_Y=np.array(Prob_Y)  #将Prob_Y改为numpy数组形式
    Prob_Y=Prob_Y.reshape(173,2)  #原本是(173,1,2)形状，改为（173,2）形状
    Prob_Y=pd.DataFrame(Prob_Y)
    Prob_Y.columns=["0","1"]
    confusion_matrix(Really_Y,Predict_Y)
    TN=confusion_matrix(Really_Y,Predict_Y)[0,0]
    FN=confusion_matrix(Really_Y,Predict_Y)[1,0]
    FP=confusion_matrix(Really_Y,Predict_Y)[0,1]
    TP=confusion_matrix(Really_Y,Predict_Y)[1,1]
    ACC=accuracy_score(Really_Y,Predict_Y)
    F1_score=f1_score(Really_Y,Predict_Y)
    AUC=roc_auc_score(Really_Y,np.array(Prob_Y["1"]))
    Recall=recall_score(Really_Y,Predict_Y)
    Precision=precision_score(Really_Y,Predict_Y)
    S=p1
    D=p2
    M=grid_search.best_params_["max_features"]
#     print("ACC:",ACC)
#     print("f1_score:",F1_score)
#     print("AUC:",AUC)
    return TN,FN,FP,TP,ACC,Recall,Precision,F1_score,AUC,S,D,M

In [7]:
file_path = 'E:\datatest\PseDPC\d6\d61'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
611.csv
**************************************************
2022-02-24 17:10:37.853211
2022-02-24 17:10:42.113809
2022-02-24 17:11:02.608974
##################################################
612.csv
**************************************************
2022-02-24 17:11:11.374522
2022-02-24 17:11:15.176351
2022-02-24 17:11:33.572134
##################################################
613.csv
**************************************************
2022-02-24 17:11:43.297113
2022-02-24 17:11:46.968291
2022-02-24 17:12:05.125711
##################################################
614.csv
**************************************************
2022-02-24 17:12:15.190782
2022-02-24 17:12:18.692413
2022-02-24 17:12:36.215530
##################################################
615.csv
**************************************************
2022-02-24 17:12:44.434540
2022-02-24 17:12:48.099734
2022-02-24 17:12:54.598347
#############################################

In [8]:
file_path = 'E:\datatest\PseDPC\d6\d62'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
621.csv
**************************************************
2022-02-24 17:15:14.478617
2022-02-24 17:15:17.949331
2022-02-24 17:15:35.954158
##################################################
622.csv
**************************************************
2022-02-24 17:15:43.932812
2022-02-24 17:15:47.435441
2022-02-24 17:16:04.740142
##################################################
623.csv
**************************************************
2022-02-24 17:16:13.067863
2022-02-24 17:16:16.922548
2022-02-24 17:16:22.939450
##################################################
624.csv
**************************************************
2022-02-24 17:16:25.808773
2022-02-24 17:16:29.462996
2022-02-24 17:16:47.789962
##################################################
625.csv
**************************************************
2022-02-24 17:16:57.510955
2022-02-24 17:17:01.244963
2022-02-24 17:17:20.472520
#############################################

In [9]:
file_path = 'E:\datatest\PseDPC\d6\d63'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
631.csv
**************************************************
2022-02-24 17:19:35.343673
2022-02-24 17:19:38.867246
2022-02-24 17:19:55.305267
##################################################
632.csv
**************************************************
2022-02-24 17:20:03.161248
2022-02-24 17:20:06.567136
2022-02-24 17:20:22.856554
##################################################
633.csv
**************************************************
2022-02-24 17:20:32.465843
2022-02-24 17:20:35.809898
2022-02-24 17:20:52.908150
##################################################
634.csv
**************************************************
2022-02-24 17:21:00.796046
2022-02-24 17:21:04.493154
2022-02-24 17:21:22.001311
##################################################
635.csv
**************************************************
2022-02-24 17:21:31.405151
2022-02-24 17:21:35.637828
2022-02-24 17:21:55.881665
#############################################

In [10]:
file_path = 'E:\datatest\PseDPC\d6\d64'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
641.csv
**************************************************
2022-02-24 17:23:26.489786
2022-02-24 17:23:30.367412
2022-02-24 17:23:52.148173
##################################################
642.csv
**************************************************
2022-02-24 17:24:06.310282
2022-02-24 17:24:10.047283
2022-02-24 17:24:30.039794
##################################################
643.csv
**************************************************
2022-02-24 17:24:40.451936
2022-02-24 17:24:45.227163
2022-02-24 17:25:08.706341
##################################################
644.csv
**************************************************
2022-02-24 17:25:21.415339
2022-02-24 17:25:26.047944
2022-02-24 17:25:47.725948
##################################################
645.csv
**************************************************
2022-02-24 17:25:57.405049
2022-02-24 17:26:01.872097
2022-02-24 17:26:25.122911
#############################################

In [11]:
file_path = 'E:\datatest\PseDPC\d6\d65'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
651.csv
**************************************************
2022-02-24 17:29:21.014314
2022-02-24 17:29:25.138283
2022-02-24 17:29:45.734176
##################################################
652.csv
**************************************************
2022-02-24 17:29:57.406946
2022-02-24 17:30:01.540886
2022-02-24 17:30:23.829254
##################################################
653.csv
**************************************************
2022-02-24 17:30:38.977724
2022-02-24 17:30:43.421833
2022-02-24 17:31:06.534994
##################################################
654.csv
**************************************************
2022-02-24 17:31:18.139945
2022-02-24 17:31:22.664840
2022-02-24 17:31:49.307556
##################################################
655.csv
**************************************************
2022-02-24 17:31:59.170169
2022-02-24 17:32:04.130897
2022-02-24 17:32:28.159608
#############################################

In [12]:
file_path = 'E:\datatest\PseDPC\d6\d66'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
661.csv
**************************************************
2022-02-24 17:35:11.826717
2022-02-24 17:35:15.942708
2022-02-24 17:35:36.976430
##################################################
662.csv
**************************************************
2022-02-24 17:35:50.569572
2022-02-24 17:35:54.960823
2022-02-24 17:36:18.879829
##################################################
663.csv
**************************************************
2022-02-24 17:36:29.350813
2022-02-24 17:36:34.462141
2022-02-24 17:36:56.600906
##################################################
664.csv
**************************************************
2022-02-24 17:37:07.308260
2022-02-24 17:37:12.012673
2022-02-24 17:37:19.108686
##################################################
665.csv
**************************************************
2022-02-24 17:37:22.583392
2022-02-24 17:37:27.461339
2022-02-24 17:37:35.002164
#############################################

In [13]:
file_path = 'E:\datatest\PseDPC\d6\d67'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
671.csv
**************************************************
2022-02-24 17:39:31.000810
2022-02-24 17:39:34.925312
2022-02-24 17:39:41.339150
##################################################
672.csv
**************************************************
2022-02-24 17:39:45.921888
2022-02-24 17:39:50.237342
2022-02-24 17:40:11.826583
##################################################
673.csv
**************************************************
2022-02-24 17:40:24.456788
2022-02-24 17:40:28.803160
2022-02-24 17:40:50.813272
##################################################
674.csv
**************************************************
2022-02-24 17:41:02.478062
2022-02-24 17:41:06.921176
2022-02-24 17:41:27.756430
##################################################
675.csv
**************************************************
2022-02-24 17:41:38.917569
2022-02-24 17:41:43.063477
2022-02-24 17:42:03.382114
#############################################

In [14]:
file_path = 'E:\datatest\PseDPC\d6\d68'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
681.csv
**************************************************
2022-02-24 17:44:40.272356
2022-02-24 17:44:44.499046
2022-02-24 17:45:04.971273
##################################################
682.csv
**************************************************
2022-02-24 17:45:16.590187
2022-02-24 17:45:20.614423
2022-02-24 17:45:40.041444
##################################################
683.csv
**************************************************
2022-02-24 17:45:49.555987
2022-02-24 17:45:53.547459
2022-02-24 17:46:14.308761
##################################################
684.csv
**************************************************
2022-02-24 17:46:23.766457
2022-02-24 17:46:27.858509
2022-02-24 17:46:48.264915
##################################################
685.csv
**************************************************
2022-02-24 17:46:59.759160
2022-02-24 17:47:04.226208
2022-02-24 17:47:25.967040
#############################################

In [15]:
file_path = 'E:\datatest\PseDPC\d6\d69'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
691.csv
**************************************************
2022-02-24 17:49:57.907525
2022-02-24 17:50:01.715337
2022-02-24 17:50:20.270692
##################################################
692.csv
**************************************************
2022-02-24 17:50:31.314145
2022-02-24 17:50:35.562777
2022-02-24 17:50:57.692570
##################################################
693.csv
**************************************************
2022-02-24 17:51:06.674541
2022-02-24 17:51:11.059805
2022-02-24 17:51:33.432946
##################################################
694.csv
**************************************************
2022-02-24 17:51:42.681203
2022-02-24 17:51:47.077441
2022-02-24 17:52:10.135749
##################################################
695.csv
**************************************************
2022-02-24 17:52:22.948468
2022-02-24 17:52:27.205079
2022-02-24 17:52:34.058743
#############################################

In [16]:
file_path = 'E:\datatest\PseDPC\d6\d610'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
6101.csv
**************************************************
2022-02-24 17:54:56.251307
2022-02-24 17:55:00.565764
2022-02-24 17:55:21.567574
##################################################
6102.csv
**************************************************
2022-02-24 17:55:31.915886
2022-02-24 17:55:35.974029
2022-02-24 17:55:55.978507
##################################################
6103.csv
**************************************************
2022-02-24 17:56:07.420898
2022-02-24 17:56:11.640603
2022-02-24 17:56:32.806973
##################################################
6104.csv
**************************************************
2022-02-24 17:56:41.515672
2022-02-24 17:56:45.746356
2022-02-24 17:57:06.990515
##################################################
6105.csv
**************************************************
2022-02-24 17:57:15.671290
2022-02-24 17:57:19.921917
2022-02-24 17:57:41.170068
########################################

In [17]:
file_path = 'E:\datatest\PseDPC\d6\d611'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
6111.csv
**************************************************
2022-02-24 18:00:14.045051
2022-02-24 18:00:17.967557
2022-02-24 18:00:37.930147
##################################################
6112.csv
**************************************************
2022-02-24 18:00:49.892143
2022-02-24 18:00:54.157730
2022-02-24 18:01:00.535666
##################################################
6113.csv
**************************************************
2022-02-24 18:01:03.704189
2022-02-24 18:01:07.969777
2022-02-24 18:01:14.659880
##################################################
6114.csv
**************************************************
2022-02-24 18:01:17.846352
2022-02-24 18:01:22.161806
2022-02-24 18:01:28.856894
##################################################
6115.csv
**************************************************
2022-02-24 18:01:32.048354
2022-02-24 18:01:36.211221
2022-02-24 18:01:57.198070
########################################

In [18]:
file_path = 'E:\datatest\PseDPC\d6\d612'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
6121.csv
**************************************************
2022-02-24 18:04:34.494222
2022-02-24 18:04:39.063995
2022-02-24 18:05:02.283871
##################################################
6122.csv
**************************************************
2022-02-24 18:05:12.287134
2022-02-24 18:05:16.743185
2022-02-24 18:05:23.586875
##################################################
6123.csv
**************************************************
2022-02-24 18:05:27.731785
2022-02-24 18:05:32.358406
2022-02-24 18:05:54.906089
##################################################
6124.csv
**************************************************
2022-02-24 18:06:05.841988
2022-02-24 18:06:10.272139
2022-02-24 18:06:32.726030
##################################################
6125.csv
**************************************************
2022-02-24 18:06:46.080323
2022-02-24 18:06:50.671020
2022-02-24 18:06:57.538644
########################################

In [19]:
file_path = 'E:\datatest\PseDPC\d6\d613'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
6131.csv
**************************************************
2022-02-24 18:08:46.112668
2022-02-24 18:08:50.779181
2022-02-24 18:09:13.915280
##################################################
6132.csv
**************************************************
2022-02-24 18:09:27.330388
2022-02-24 18:09:32.626220
2022-02-24 18:09:58.627653
##################################################
6133.csv
**************************************************
2022-02-24 18:10:08.468324
2022-02-24 18:10:14.299722
2022-02-24 18:10:38.804161
##################################################
6134.csv
**************************************************
2022-02-24 18:10:48.222962
2022-02-24 18:10:53.296415
2022-02-24 18:11:17.020971
##################################################
6135.csv
**************************************************
2022-02-24 18:11:26.304075
2022-02-24 18:11:30.858891
2022-02-24 18:11:37.423326
########################################

In [20]:
file_path = 'E:\datatest\PseDPC\d6\d614'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
6141.csv
**************************************************
2022-02-24 18:12:39.563076
2022-02-24 18:12:44.441020
2022-02-24 18:13:07.615018
##################################################
6142.csv
**************************************************
2022-02-24 18:13:16.803434
2022-02-24 18:13:21.772141
2022-02-24 18:13:48.000004
##################################################
6143.csv
**************************************************
2022-02-24 18:13:59.449335
2022-02-24 18:14:04.059002
2022-02-24 18:14:27.733147
##################################################
6144.csv
**************************************************
2022-02-24 18:14:36.947488
2022-02-24 18:14:41.583064
2022-02-24 18:15:05.084187
##################################################
6145.csv
**************************************************
2022-02-24 18:15:15.034671
2022-02-24 18:15:19.847688
2022-02-24 18:15:44.191558
########################################

In [21]:
file_path = 'E:\datatest\PseDPC\d6\d615'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
6151.csv
**************************************************
2022-02-24 18:18:13.559950
2022-02-24 18:18:17.855429
2022-02-24 18:18:39.847589
##################################################
6152.csv
**************************************************
2022-02-24 18:18:48.577268
2022-02-24 18:18:52.979455
2022-02-24 18:19:15.104287
##################################################
6153.csv
**************************************************
2022-02-24 18:19:23.890752
2022-02-24 18:19:28.265049
2022-02-24 18:19:34.293918
##################################################
6154.csv
**************************************************
2022-02-24 18:19:38.434839
2022-02-24 18:19:42.844070
2022-02-24 18:19:48.880922
##################################################
6155.csv
**************************************************
2022-02-24 18:19:53.094662
2022-02-24 18:19:57.371210
2022-02-24 18:20:03.427009
########################################

In [22]:
file_path = 'E:\datatest\PseDPC\d6\d616'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
6161.csv
**************************************************
2022-02-24 18:21:03.775546
2022-02-24 18:21:07.821721
2022-02-24 18:21:28.367751
##################################################
6162.csv
**************************************************
2022-02-24 18:21:38.963403
2022-02-24 18:21:43.516221
2022-02-24 18:22:05.829523
##################################################
6163.csv
**************************************************
2022-02-24 18:22:14.561160
2022-02-24 18:22:18.964382
2022-02-24 18:22:41.040314
##################################################
6164.csv
**************************************************
2022-02-24 18:22:52.605400
2022-02-24 18:22:57.003605
2022-02-24 18:23:19.029676
##################################################
6165.csv
**************************************************
2022-02-24 18:23:27.779265
2022-02-24 18:23:31.763635
2022-02-24 18:23:51.654406
########################################

In [23]:
file_path = 'E:\datatest\PseDPC\d6\d617'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
6171.csv
**************************************************
2022-02-24 18:26:10.311411
2022-02-24 18:26:14.283784
2022-02-24 18:26:34.250362
##################################################
6172.csv
**************************************************
2022-02-24 18:26:45.059443
2022-02-24 18:26:49.548433
2022-02-24 18:27:11.805883
##################################################
6173.csv
**************************************************
2022-02-24 18:27:22.681822
2022-02-24 18:27:27.181776
2022-02-24 18:27:49.463131
##################################################
6174.csv
**************************************************
2022-02-24 18:28:00.314099
2022-02-24 18:28:04.813063
2022-02-24 18:28:27.082481
##################################################
6175.csv
**************************************************
2022-02-24 18:28:37.930492
2022-02-24 18:28:42.411503
2022-02-24 18:29:04.546303
########################################

In [24]:
file_path = 'E:\datatest\PseDPC\d6\d618'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
6181.csv
**************************************************
2022-02-24 18:31:42.533587
2022-02-24 18:31:46.834087
2022-02-24 18:32:08.073284
##################################################
6182.csv
**************************************************
2022-02-24 18:32:17.302131
2022-02-24 18:32:22.158140
2022-02-24 18:32:46.133991
##################################################
6183.csv
**************************************************
2022-02-24 18:32:58.243592
2022-02-24 18:33:02.895147
2022-02-24 18:33:25.456797
##################################################
6184.csv
**************************************************
2022-02-24 18:33:36.547112
2022-02-24 18:33:41.124865
2022-02-24 18:34:03.949796
##################################################
6185.csv
**************************************************
2022-02-24 18:34:14.843792
2022-02-24 18:34:19.517146
2022-02-24 18:34:42.196467
########################################

In [25]:
file_path = 'E:\datatest\PseDPC\d6\d619'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
6191.csv
**************************************************
2022-02-24 18:37:26.743480
2022-02-24 18:37:31.088854
2022-02-24 18:37:52.879554
##################################################
6192.csv
**************************************************
2022-02-24 18:38:05.148727
2022-02-24 18:38:09.525020
2022-02-24 18:38:31.134204
##################################################
6193.csv
**************************************************
2022-02-24 18:38:43.794331
2022-02-24 18:38:48.222485
2022-02-24 18:39:09.584382
##################################################
6194.csv
**************************************************
2022-02-24 18:39:22.283405
2022-02-24 18:39:26.591878
2022-02-24 18:39:49.122984
##################################################
6195.csv
**************************************************
2022-02-24 18:40:01.760174
2022-02-24 18:40:06.525425
2022-02-24 18:40:30.591036
########################################

In [26]:
file_path = 'E:\datatest\PseDPC\d6\d620'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
6201.csv
**************************************************
2022-02-24 18:43:14.693982
2022-02-24 18:43:18.709238
2022-02-24 18:43:25.447209
##################################################
6202.csv
**************************************************
2022-02-24 18:43:29.575166
2022-02-24 18:43:33.797869
2022-02-24 18:43:40.542824
##################################################
6203.csv
**************************************************
2022-02-24 18:43:43.627570
2022-02-24 18:43:47.782452
2022-02-24 18:43:54.112516
##################################################
6204.csv
**************************************************
2022-02-24 18:43:58.035021
2022-02-24 18:44:02.116102
2022-02-24 18:44:08.424225
##################################################
6205.csv
**************************************************
2022-02-24 18:44:12.689813
2022-02-24 18:44:16.837715
2022-02-24 18:44:23.138856
########################################